# Initiate

In [ ]:
print('Initiate in process ...')

from Import_pakages import *
import Import_functions as Import_functions
import Conversion_functions as Conversion_functions
import Vis_functions as Vis_functions

from pathlib import Path
output_filepath = '/Users/'  # The file path for the output files

%matplotlib widget
%load_ext autoreload
%autoreload 2

print('Initiate complete')

# Some  clarification

The three axes of the 3D data matrix are set in the order of [Emission angle (Y), Binding energy (eb), Photon energy (hv)]

Symbols:

* eb: Binding Energy (eV) 
* Y: Emission Angle (Deg)
* tilt: Tilt Angle (Deg)
* kx: $k_x$ (Å$^{-1})$
* ky: $k_y$ (Å$^{-1})$
* E_work: Work Function, defaut is set to 4.42 (eV)
* T: Temperature (K)

# Parameters Input + Normalization to data1

## Improt from .h5 file

In [ ]:
filepath = '/Users/Sample_data.h5'

sample_name = ['20251225', 'Sample']

data0, axes, hv, E_work, T = Import_functions.import_h5py(filepath)

# 4. Generate Coordinates
eb = -axes[0]
Y = axes[1]
tilt = axes[2]
    
# 5. Efficient Normalization
# Compute mean along spatial axes (0 and 1)
# This is much faster if done before swapping axes
in_sum = np.mean(data0, axis=(0, 1))
data1 = data0 / in_sum[np.newaxis, np.newaxis, :]

print('Importing completed')

In [ ]:
print(np.shape(data1))
print(np.shape(eb))
print(np.shape(Y))
print(np.shape(tilt))

# Binding Energy Raw Plot

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data0, axes=[eb, Y, tilt], sample_name=sample_name, output_filepath=output_filepath)
plt.show()

In [ ]:
Y = Y+0.5

# Fermi Edge Alignment data2 ($Y$, $E_b$, hv)

## Fermi Edge Fitting

### Integration over emission angles

In [ ]:
# Binding Energy Range for Fermi Edge Fitting
eb_range = [-0.15, 0.2]

# Matrix for Intensity data integrated over angles
data1_angle_sum = np.sum(data1, axis=0)

plt.close()
Vis_functions.EDC_angle_sum(data=data1_angle_sum, eb=eb, z=hv, eb_range=eb_range, sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### Fermi-Dirac Model Fitting

In [ ]:
fit_range = [0.15, -0.15] # The data range in index of binding energy for fitting

fermi_fit_results = Conversion_functions.Fermi_level_fit_fd(data1_angle_sum, fit_range, eb, hv, T)

Fermi_fit_dict = {'offset': fermi_fit_results[:, 0], 
                  'Temp': fermi_fit_results[:, 1],
                  'fermi_edge_amplitude': fermi_fit_results[:, 2],
                  'fermi_edge_const_bkg': fermi_fit_results[:, 3],
                  'fermi_edge_lin_bkg': fermi_fit_results[:, 4] }

print('Fitting Completed')
print('R-squared:')
print(fermi_fit_results[:,-1] )

In [ ]:
plt.close()
Vis_functions.Fermi_edge_fit_plot(data=data1_angle_sum, eb=eb, hv=hv, fit_range=fit_range,
                                               fermi_edge_center=fermi_fit_results[:, 0], fermi_edge_T= fermi_fit_results[:, 1],
                                               fermi_edge_amplitude=fermi_fit_results[:, 2], fermi_edge_const_bkg=fermi_fit_results[:, 3], fermi_edge_lin_bkg=fermi_fit_results[:, 4],
                                               sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### Affine_broadened_fd Model Fitting

In [ ]:
fit_range = [0.15, -0.15] # The data range in index of binding energy for fitting

fermi_fit_results = Conversion_functions.Fermi_level_fit_ABfd(data1_angle_sum, fit_range, eb, tilt, T, 
                                                              center=[0.025, 0, 0.05] )

Fermi_fit_dict = {'offset': fermi_fit_results[:, 0], 
                  'Temp': fermi_fit_results[:, 1],
                  'fermi_edge_conv_width': fermi_fit_results[:, 2],
                  'fermi_edge_const_bkg': fermi_fit_results[:, 3],
                  'fermi_edge_lin_bkg': fermi_fit_results[:, 4],
                  'BG_offset': fermi_fit_results[:, 5]}

print('Fitting Completed')
print('R-squared:')
print(fermi_fit_results[:,-1] )

In [ ]:
fermi_edge_center_index = find_value_index(eb, fermi_fit_results[:, 0])

plt.close()
fig, ax1, ax2, ax3= Vis_functions.Affine_broadened_Fermi_edge_fit_plot(data=data1_angle_sum, eb=eb, hv=tilt, fit_range=fit_range,
                                               fermi_edge_center=fermi_fit_results[:, 0], fermi_edge_T= fermi_fit_results[:, 1], fermi_edge_conv_width=fermi_fit_results[:, 2], 
                                               fermi_edge_const_bkg=fermi_fit_results[:, 3], fermi_edge_lin_bkg=fermi_fit_results[:, 4], fermi_edge_offset=fermi_fit_results[:, 5], 
                                               sample_name=sample_name, output_filepath=output_filepath)
plt.show()

## Fit Offsets with Linear Function

In [ ]:
fermi_edge_center_index = find_value_index(eb, fermi_fit_results[:, 0])

#Linear fitting of Fermi edge center vs hv for alignment
def linear_func(x, slope, c): 
    return slope * x + c
mod = lmfit.Model(linear_func)
params = mod.make_params(slope=1, c=0)
out = mod.fit(fermi_fit_results[:, 0], params, x=tilt, method='nelder')
print(out.fit_report())
fermi_edge_center_index = find_value_index(eb, linear_func(tilt, out.params['slope'].value, out.params['c'].value))

Fermi_fit_dict['orange'] = 3

plt.close()
fig, ax = plt.subplots(1, 1, figsize=(5,2), layout='constrained')

print(np.shape(fermi_fit_results[:, 0]))

ax.plot(tilt, fermi_fit_results[:, 0], 'o', label='Data', markersize=4)
ax.plot(tilt, linear_func(tilt, out.params['slope'].value, out.params['c'].value), '-', label='Fit', markersize=4)
plt.show()

## Alignment & output

In [ ]:
print('Alignment in process ...')

Eb_0_index = np.abs(eb - 0).argmin()
# 1. Pre-calculate shifts for each k
shifts = np.round(Eb_0_index - fermi_edge_center_index).astype(int)
# 2. Vectorized approach
data2 = np.zeros_like(data1)
for k, shift in enumerate(shifts):
    # Process the entire [Y, eb] slice for this 'k' at once
    if shift > 0:
        data2[shift:, :, k] = data1[:-shift, :, k]
    elif shift < 0:
        data2[:shift, :, k] = data1[-shift:, :, k]
    else:
        data2[:, :, k] = data1[:, :, k]
print('Alignment completed')

matlab_mat = {"data": data2, 'Eb': eb, 'Y': Y, 'Tilt': tilt, 'hv': hv}
matlab_mat =  matlab_mat | Fermi_fit_dict

file_path = Path(output_filepath+sample_name[1]+'_Eb_Y_Tilt.mat')
if not file_path.exists():
    savemat(file_path, matlab_mat)
    print(f"Saved to {file_path}")
else:
    print( '\033[31m' + f"File {file_path} already exists. Skipping save." + '\033[0m' )

## Alignment check

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data2, axes=[eb, Y, tilt], sample_name=sample_name, output_filepath=output_filepath)
plt.show()

### EDCs averaging over emission angles

In [ ]:
data2_angle_sum = np.sum(data2, axis=1)
        
eb_range = [-0.1, 0.15]

plt.close()
Vis_functions.EDC_angle_sum(data=data2_angle_sum, eb=eb, z=tilt, eb_range=eb_range, sample_name=sample_name, output_filepath=output_filepath)
plt.show()

# Convert Angles to $k$ -- data3 ($E_b$, $k_x$, $k_y$)

## For photon energy scans to data3

In [ ]:
k_res_factor = 1
kx, ky, data3 = Conversion_functions.convert_angle_to_k_map(
    data=data2, 
    eb_axis=eb,
    angle_axis=Y, 
    tilt_axis=tilt, 
    hv=hv,
    work_function=E_work,
    k_res_factor=k_res_factor
)

matlab_mat = {"data": data3, 'Eb': eb, 'kx': kx, 'ky': ky, 'hv': hv, 'E_work': E_work, 'k_res_factor': k_res_factor}
matlab_mat = matlab_mat | Fermi_fit_dict

file_path = Path(output_filepath+sample_name[1]+'Eb_kx_ky_interp.mat')
if not file_path.exists():
    savemat(file_path, matlab_mat)
    print(f"Saved to {file_path}")
else:
    print( '\033[31m' + f"File {file_path} already exists. Skipping save." + '\033[0m' )

## Visualization

In [ ]:
plt.close()
fig, ax = Vis_functions.slice_3D(data=data3, axes=[eb, kx, ky], sample_name=sample_name, output_filepath=output_filepath)
plt.show()